In [1]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [2]:
# Fixed parameters
filename = "T0n09pip_full_new_weak.csv"
file = open(filename,"w+")
K = 0.0
U = 1.7
V = -2.3
Temp = 0.001
nval = 0.9
Lsmall = 12
Llarge = 40
# Fixed functions
def eps(kx,ky):
    f = -2.0*(np.cos(kx) + np.cos(ky))
    return f

def sk(kx,ky):
    f = 1/2*(np.cos(kx) + np.cos(ky))
    return f

def dk(kx,ky):
    f = 1/2*(np.cos(kx) - np.cos(ky))
    return f

def deltaTrip(kx,ky,dx,dy):
    trip = (1.0j)*dx*np.sin(kx) + dy*np.sin(ky)
    return trip

def energy(mu,kx,ky,dx,dy):
    f = np.sqrt((eps(kx, ky) - mu)**2 + np.abs(deltaTrip(kx,ky,dx,dy))**2)
    return f

def fermi(T,mu,kx,ky,dx,dy):
    f = 1.0/(np.exp(energy(mu,kx,ky,dx,dy)/T) + 1.0)
    return f

def g(T,mu,kx,ky,dx,dy):
    f = (1.0 - 2.0*fermi(T,mu,kx,ky,dx,dy))/(2.0*energy(mu,kx,ky,dx,dy)+0.0000001)
    return f

In [3]:
while V<-0.2:
    U = -6.0
    while U<2.5:#Temp<Tempend:
        print('nval =', nval,' Temp =', Temp, ' V = ',V)
        L = Lsmall
        NSites = L**2
        delK = 2.0*np.pi/np.sqrt(NSites)
        kvals = [-np.pi + delK + i*delK for i in range(L)]

        def n(mu,T,dx,dy):
            sump = 2.0/(NSites)*sum((eps(kvals[i],kvals[j])-mu)*g(T,mu,kvals[i],kvals[j],dx,dy) for i in range(len(kvals)) for j in range(len(kvals)))
            f = 1.0-sump
            return f

        xtab = np.zeros((len(kvals),len(kvals)))
        ytab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                xtab[i,j] = np.sin(kvals[i])
                ytab[i,j] = np.sin(kvals[j])

        def numeqn(mu,dx,dy):
            return nval - n(mu,Temp,dx,dy)

        def FreeE(p):
            Delx, Dely = p 

            mu = optimize.root(numeqn, -1.5, args=(Delx,Dely), method='broyden1', options={'ftol':0.01}).x

            gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delTripp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            ixx = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iyy = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            for i in range(len(kvals)):
                for j in range(len(kvals)):
                    gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Delx, Dely)
                    lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Delx, Dely))
                    epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Delx,Dely)
                    delTripp[i,j] = deltaTrip(kvals[i],kvals[j],Delx,Dely)
                    del2p[i,j] = (np.abs(delTripp[i,j])**2)*gp[i,j]
                    ixx[i,j] = xtab[i,j]*(delTripp[i,j]*gp[i,j])
                    iyy[i,j] = ytab[i,j]*(delTripp[i,j]*gp[i,j])

            Ixx = np.abs(1.0/(NSites)*np.sum(ixx))**2
            Iyy = np.abs(1.0/(NSites)*np.sum(iyy))**2

            logterm = 2.0*Temp/NSites*(np.sum(lnp))
            enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
            Vterm = 2.0*V*Ixx + 2.0*V*Iyy

            return np.real(enerterm + logterm + mu*nval + Vterm) 

        freeEval = 10.0;
        for i in range(24):
            r1 = np.random.uniform(0.06,3.0)
            r2 = r1
            f = FreeE((r1,r2))
            if f<freeEval:
                freeEval = f
                globmin = np.array([r1,r2])
                print(globmin,f)

        #globminp=optimize.minimize(FreeE, globmin, callback=None, options={'gtol': 2e-3, 'disp': True}).x
        globminp1=np.array([globmin[0],globmin[1]])
        globminp1=optimize.minimize(FreeE, globminp1, callback=None, options={'gtol': 8e-3, 'disp': True}).x
        freeEpval1=FreeE(globminp1)

        freeEval = 10.0;
        for i in range(13):
            r2 = np.random.uniform(-3.0,-0.04)
            r1 = -r2
            f = FreeE((r1,r2))
            if f<freeEval:
                freeEval = f
                globmin = np.array([r1,r2])
                print(globmin,f)

        #globminp=optimize.minimize(FreeE, globmin, callback=None, options={'gtol': 2e-3, 'disp': True}).x
        globminp2=np.array([-globmin[1],globmin[1]])
        globminp2=optimize.minimize(FreeE, globminp2, callback=None, options={'gtol': 4e-3, 'disp': True}).x
        freeEpval2=FreeE(globminp2)


        freelist=np.array([freeEpval1,freeEpval2])
        globminlist=np.array(np.array([globminp1,globminp2]))
        result = np.where(freelist == np.amin(freelist))
        globmin = globminlist[result[0]][0]
        print(globmin, " ", np.amin(freelist))

        # Fixed parameters for larger lattice
        L = Llarge
        NSites = L**2
        delK = 2.0*np.pi/np.sqrt(NSites)
        kvals = [-np.pi + delK + i*delK for i in range(L)]

        def n(mu,T,dx,dy):
            sump = 2.0/(NSites)*sum((eps(kvals[i],kvals[j])-mu)*g(T,mu,kvals[i],kvals[j],dx,dy) for i in range(len(kvals)) for j in range(len(kvals)))
            f = 1.0-sump
            return f

        xtab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                xtab[i,j] = (np.sin(kvals[i]))

        ytab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                ytab[i,j] = (np.sin(kvals[j]))

        def FreeE(p):
            Delx, Dely = p 

            mu = optimize.root(numeqn, -1.5, args=(Delx,Dely), method='broyden1', options={'ftol':0.008}).x

            gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delTripp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            ixx = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iyy = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            for i in range(len(kvals)):
                for j in range(len(kvals)):
                    gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Delx, Dely)
                    lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Delx, Dely))
                    epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Delx,Dely)
                    delTripp[i,j] = deltaTrip(kvals[i],kvals[j],Delx,Dely)
                    del2p[i,j] = (np.abs(delTripp[i,j])**2)*gp[i,j]
                    ixx[i,j] = xtab[i,j]*(delTripp[i,j]*gp[i,j])
                    iyy[i,j] = ytab[i,j]*(delTripp[i,j]*gp[i,j])

            Ixx = np.abs(1.0/(NSites)*np.sum(ixx))**2
            Iyy = np.abs(1.0/(NSites)*np.sum(iyy))**2

            logterm = 2.0*Temp/NSites*(np.sum(lnp))
            enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
            Vterm = 2.0*V*Ixx + 2.0*V*Iyy

            return np.real(enerterm + logterm + mu*nval + Vterm)

        globmin=optimize.minimize(FreeE, globmin, callback=None, options={'gtol': 3e-6, 'disp': True}).x
        F=FreeE(globmin)

        mu = optimize.root(numeqn, -1.5, args=(globmin[0],globmin[1]), method='broyden1', options={'ftol':0.008}).x
        data = (U, V, mu, globmin[0], globmin[1], F)
        print(data)
        
    #U = -6.0
    #while U < 4.0:
    #    data = (U,V, globmin[0], globmin[1], F)
    #    file.write("%s,%s,%s,%s,%s\n" % data)
    #    U = U + 0.1
        file.write("%s,%s,%s,%s,%s,%s\n" % data)
        
        U = U + 0.1
    
    V = V + 0.1
    
file.close()

nval = 0.9  Temp = 0.001  V =  -2.3
[2.03877482 2.03877482] -1.5061071379023403
[1.3045296 1.3045296] -1.5857373549107907
[0.90081586 0.90081586] -1.6200468748961578
[0.7406388 0.7406388] -1.629263388061661
[0.39428774 0.39428774] -1.634765500565175
Optimization terminated successfully.
         Current function value: -1.635288
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 1.53129242 -1.53129242] -1.5623825362304142
[ 0.58485304 -0.58485304] -1.634586035835951
Optimization terminated successfully.
         Current function value: -1.635650
         Iterations: 2
         Function evaluations: 16
         Gradient evaluations: 4
[ 0.47998026 -0.47998073]   -1.635649574532329
Optimization terminated successfully.
         Current function value: -1.635801
         Iterations: 4
         Function evaluations: 20
         Gradient evaluations: 5
(-6.0, -2.3, array(-0.19942091), 0.48110970435794254, -0.48111000772084284, -1.6358008753062316)
nv

[0.44902498 0.44902498] -1.6355214762673134
[0.45716327 0.45716327] -1.6355756741999332
Optimization terminated successfully.
         Current function value: -1.635576
         Iterations: 0
         Function evaluations: 4
         Gradient evaluations: 1
[ 1.94744539 -1.94744539] -1.5163979379492234
[ 1.00203963 -1.00203963] -1.612690092279063
[ 0.25239939 -0.25239939] -1.6294867400090076
[ 0.44973603 -0.44973603] -1.6355268014760997
Optimization terminated successfully.
         Current function value: -1.635527
         Iterations: 0
         Function evaluations: 4
         Gradient evaluations: 1
[0.45716327 0.45716327]   -1.6355756741999332
Optimization terminated successfully.
         Current function value: -1.635801
         Iterations: 4
         Function evaluations: 20
         Gradient evaluations: 5
(-5.200000000000003, -2.3, array(-0.19941952), 0.4810880946261946, 0.4810878653983269, -1.6358008752556177)
nval = 0.9  Temp = 0.001  V =  -2.3
[2.37096506 2.37096506] -1.4

(-4.500000000000005, -2.3, array(-0.19942048), 0.4811027948579129, -0.48110341722664857, -1.635800875301576)
nval = 0.9  Temp = 0.001  V =  -2.3
[1.25600261 1.25600261] -1.590452057807841
[1.05790051 1.05790051] -1.6082144190688574
[1.04341118 1.04341118] -1.609401186284733
[0.87858708 0.87858708] -1.621516834072454
[0.43855317 0.43855317] -1.6354298860831766
Optimization terminated successfully.
         Current function value: -1.635430
         Iterations: 0
         Function evaluations: 4
         Gradient evaluations: 1
[ 0.07973772 -0.07973772] -1.6175776798057802
[ 0.30237433 -0.30237433] -1.6319009443612704
[ 0.45606561 -0.45606561] -1.6355692255670502
Optimization terminated successfully.
         Current function value: -1.635569
         Iterations: 0
         Function evaluations: 4
         Gradient evaluations: 1
[ 0.45606561 -0.45606561]   -1.6355692255670502
Optimization terminated successfully.
         Current function value: -1.635801
         Iterations: 4
        

[ 0.60036896 -0.60036896] -1.6342447428256623
[ 0.42575294 -0.42575294] -1.6352841781150333
Optimization terminated successfully.
         Current function value: -1.635649
         Iterations: 3
         Function evaluations: 16
         Gradient evaluations: 4
[ 0.47914834 -0.47914832]   -1.6356488789385768
Optimization terminated successfully.
         Current function value: -1.635801
         Iterations: 4
         Function evaluations: 20
         Gradient evaluations: 5
(-3.6000000000000068, -2.3, array(-0.19942088), 0.48110937078901495, -0.48110940376219674, -1.6358008753062254)
nval = 0.9  Temp = 0.001  V =  -2.3
[0.06008142 0.06008142] -1.6160196207368718
[0.79600887 0.79600887] -1.6264567640782885
[0.50057511 0.50057511] -1.6356193511166464
Optimization terminated successfully.
         Current function value: -1.635619
         Iterations: 0
         Function evaluations: 4
         Gradient evaluations: 1
[ 2.76649499 -2.76649499] -1.4256675198032807
[ 0.05861217 -0.058612

(-2.800000000000006, -2.3, array(-0.19942088), 0.48110923997422167, 0.4811094201341807, -1.6358008753062212)
nval = 0.9  Temp = 0.001  V =  -2.3
[2.78687724 2.78687724] -1.4234948887369685
[1.39899518 1.39899518] -1.5762428765707441
[1.25561998 1.25561998] -1.5904887431950443
[0.38213548 0.38213548] -1.6345031317956942
Optimization terminated successfully.
         Current function value: -1.635184
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 2.88036234 -2.88036234] -1.4136038795668888
[ 0.3127769 -0.3127769] -1.6323278006406754
[ 0.36092829 -0.36092829] -1.6339611477776512
Optimization terminated successfully.
         Current function value: -1.635649
         Iterations: 3
         Function evaluations: 16
         Gradient evaluations: 4
[ 0.4788983  -0.47889835]   -1.635648640633986
Optimization terminated successfully.
         Current function value: -1.635801
         Iterations: 4
         Function evaluations: 20
         Gradien

...

Optimization terminated successfully.
         Current function value: -1.604249
         Iterations: 3
         Function evaluations: 28
         Gradient evaluations: 7
[0.01843617 0.01843615]   -1.6042688137597907
Optimization terminated successfully.
         Current function value: -1.613106
         Iterations: 5
         Function evaluations: 24
         Gradient evaluations: 6
(-0.7000000000000045, -0.29999999999999893, array(-0.20187641), -2.0225607814258346e-08, -6.2587392674716e-09, -1.6131057674720062)
nval = 0.9  Temp = 0.001  V =  -0.29999999999999893
[0.27235096 0.27235096] -1.5844768421017124
[0.15356839 0.15356839] -1.5986506294295149
Optimization terminated successfully.
         Current function value: -1.604258
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 0.42775484 -0.42775484] -1.5580798319875144
[ 0.38427859 -0.38427859] -1.5660877414000154
Optimization terminated successfully.
         Current function value: -1.604

[0.13455138 0.13455138] -1.6001653442046349
Optimization terminated successfully.
         Current function value: -1.604260
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 2.35235031 -2.35235031] -1.1615029530768044
[ 1.15979915 -1.15979915] -1.3985391393211044
[ 0.31733372 -0.31733372] -1.5775317142912975
[ 0.05935951 -0.05935951] -1.6037925385016092
Optimization terminated successfully.
         Current function value: -1.604269
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 0.02007076 -0.02007079]   -1.6042689925071605
Optimization terminated successfully.
         Current function value: -1.613106
         Iterations: 5
         Function evaluations: 24
         Gradient evaluations: 6
(0.09999999999999543, -0.29999999999999893, array(-0.20187641), 1.1217128222462485e-08, -2.1863029465720937e-08, -1.6131057674720062)
nval = 0.9  Temp = 0.001  V =  -0.29999999999999893
[0.84012461 0.84012461] -

[ 0.01968977 -0.01968983]   -1.6042690923622425
Optimization terminated successfully.
         Current function value: -1.613106
         Iterations: 5
         Function evaluations: 24
         Gradient evaluations: 6
(0.7999999999999954, -0.29999999999999893, array(-0.20187641), 1.8231760967654784e-08, -6.827851769022062e-08, -1.6131057674720044)
nval = 0.9  Temp = 0.001  V =  -0.29999999999999893
[1.9984553 1.9984553] -1.2251755315129134
[1.12682642 1.12682642] -1.4058917413181358
[1.09935587 1.09935587] -1.412035844525953
[1.07243547 1.07243547] -1.4180711863168076
[0.12847586 0.12847586] -1.6005979857900263
[0.12583277 0.12583277] -1.6007785645675905
[0.12257194 0.12257194] -1.6009944137072263
Optimization terminated successfully.
         Current function value: -1.604261
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 1.27630902 -1.27630902] -1.3727928247988066
[ 1.03716457 -1.03716457] -1.4259967380283107
[ 0.64532456 -0.64532456] -1.

Optimization terminated successfully.
         Current function value: -1.604138
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 0.80090359 -0.80090359] -1.4792003004165706
[ 0.65557096 -0.65557096] -1.5113202213451338
Optimization terminated successfully.
         Current function value: -1.604252
         Iterations: 3
         Function evaluations: 20
         Gradient evaluations: 5
[-0.02709333  0.02709337]   -1.604251737866554
Optimization terminated successfully.
         Current function value: -1.613106
         Iterations: 5
         Function evaluations: 24
         Gradient evaluations: 6
(1.5999999999999959, -0.29999999999999893, array(-0.20187641), -7.76103678280229e-09, -1.7894110430438735e-08, -1.6131057674720062)
nval = 0.9  Temp = 0.001  V =  -0.29999999999999893
[0.68142807 0.68142807] -1.5056842835546922
[0.10990428 0.10990428] -1.6017656493965555
Optimization terminated successfully.
         Current function value: -1.60

[0.17505815 0.17505815] -1.596654377441884
Optimization terminated successfully.
         Current function value: -1.604266
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 3
[ 2.70663188 -2.70663188] -1.1033422299254303
[ 0.67468217 -0.67468217] -1.507158803198786
[ 0.47388241 -0.47388241] -1.5491809016233062
Optimization terminated successfully.
         Current function value: -1.604269
         Iterations: 1
         Function evaluations: 12
         Gradient evaluations: 3
[-0.01835823  0.01835838]   -1.604268765765414
Optimization terminated successfully.
         Current function value: -1.613106
         Iterations: 5
         Function evaluations: 24
         Gradient evaluations: 6
(2.3999999999999964, -0.29999999999999893, array(-0.20187641), -3.4262086908814406e-08, 2.9871749505091074e-09, -1.6131057674720055)
nval = 0.9  Temp = 0.001  V =  -0.29999999999999893
[0.35242639 0.35242639] -1.5716803013583112
[0.35017015 0.35017015] -1.5720